## Code & Logic for Determining Open Accounts

Consider the following hypothesis:

---
> $H_1$: It is possible to predict (with some metric) if an account will be closed within the next 30-days using at minimum the following feature set:

>> {
    prior 30-transactions,
    is joint account?,
    age of account owner
}
---
### STEP 2: (logical breakdown of hypothesis exploration)
> When considering $H_1$ or $H_0$, how do we find **accounts which are open (or active)**?

For the aforementioned hypothesis:
This Jupyter Notebook is concerned with finding **open (or active) accounts** only, solely, where additional separate files are to follow for subsequent steps.

#### IMPORTANT NOTE: Exploration carried out via BELLCO_PROD_DNA.OSIBANK, Bellco Credit Union's institutional data.

In [1]:
import numpy as np
import pandas as pd
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import col, lit, sum as sum_, min as min_, max as max_, count as count_
import snowflake.snowpark.functions as f
import yaml
import os

In [2]:
def create_snowpark_session():
    """
    Function to establish a Snowflake session.
    Authentication is required and accomplished via web browser where a new tab will be opened 
    which can be closed after authentication is completed.
    NOTE: 'account' and 'user' must be specified & are passed through a YAML config file in this implementation.  
    
    Parameters:
        None
    
    Returns:
        Session: a Snowflake session object.
    """
    # open yaml config file containing user, authenticator, etc.
    with open('config.yml', 'r') as file:
        config = yaml.safe_load(file)
    
    return Session.builder.configs(config).create()


def get_session_info(session):
    """
    Function to printout current Snowflake session information.
    
    Parameters:
        session (Snowflake session object): a Snowflake session object.
    
    Returns:
        None. Prints current session info.
    """
    print(f"""
{'account':<15}{session.get_current_account()}
{'role':<15}{session.get_current_role()}
{'warehouse':<15}{session.get_current_warehouse()}
{'database':<15}{session.get_current_database()}
{'schema':<15}{session.get_current_schema()}
""")
    
    
def get_view(session, view, row_lim=1, printSession=True):
    """
    Function to examine chosen VIEW including printing the number of rows found, 
    while also while printing out current session info if desired.
    
    Parameters:
        session (Snowflake session object): used to establish a connection to the desired Snowflake instance.
        view (string): the desired virtual table to examine.
        row_lim (integer): The maximum number of rows to return.
        printSession (Boolean): Should the current Snowflake session info be printed? [see function get_session_info]
    
    Returns:
        list: results from the Snowflake virtual table query are returned as a list.
    """
    if printSession:
        get_session_info(snow_session)
    
    print(f"\nrecords returned from VIEW={view}: {snow_session.table(view).count():,}")
    
    return snow_session.table(view).select("*").limit(row_lim).collect()

In [3]:
snow_session = create_snowpark_session()

# set database & schema to be used for session
snow_session.use_database("BELLCO_PROD_DNA")
snow_session.use_schema("OSIBANK")

get_session_info(snow_session)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...

account        "otscuso"
role           "OTS_INNOVATION"
warehouse      "OTS_INNOVATION_WH"
database       "BELLCO_PROD_DNA"
schema         "OSIBANK"



# Relevant Relational Database VIEWS

## `RTXN`
The Transaction table identifies all transactions.  An open account will have had member initiatiated transactions on their account for the lookback period per our definition.

### Relevant Columns for view `RTXN`

* **ACCTNBR** - The Account Number is the system assigned number that uniquely identifies each account
* **RTXNNBR** - The Transaction Number is the system assigned number used to identify each transaction.
* **PARENTACCTNBR** - The Parent Account Number is the related parent account number for an account.
* **RTXNTYPCD** - The Transaction Type Code is a user assigned code that identifies the valid transactions.
* **EXTRTXNDESCNBR** - The External Transaction Description Number is the system assigned number that uniquely identifies an external transactions description.
* **INTRRTXNDESCNBR** - The Internal Transaction Description Number is the system assigned number that uniquely identifies an internal transactions description.
* **CHECKNBR** - The Check Number is the assigned number that identifies a check.
* **TRACENBR** - The Trace Number is the user assigned number for all ACH transactions.
* **TRANAMT** - transaction dollar amount
* **ORIGPOSTDATE** - original post date
* **REVERSALRTXNNBR** - ReversalRtxnNbr identifies the transaction that reversed this item.
* **RTXNSOURCECD** - Identifies the source of a transaction.  This allows TCBS to better identify a transaction.  A deposit can be processed by many sources.  Some examples are WWW, VRU, Interest Allotments.
* **RTXNREASONCD** - Identifies the reason the transaction is processed.  This enables transactions to be processed for other reasons.  For example a reason would be Received via Mail.  The SC routines could be customized to use this.
* **OTCPERSNBR** - Person Number related to the transaction.
* **ONBEHALFOFACCTNBR** - The On Behalf Of Account Number uniquely identifies an account owned by the customer/member on whose behalf this transaction was posted.

In [4]:
%%capture
# THE FOLLOWING OUTPUT IS WITHELD DUE TO SENSITIVE Personally identifiable information (PII)

pd.set_option('display.max_columns', None)

view_obj = get_view(snow_session, view="RTXN", row_lim=100000, printSession=True)
rtxn_df = pd.DataFrame(view_obj)
print("number of columns:", len(rtxn_df.columns.values), "\ncolumns:", sorted(rtxn_df.columns.values))
rtxn_df.sort_values(['ACCTNBR', 'RTXNNBR']).head(10)

# Open Accounts [Active Accounts]
For the "not closed accounts" dataset, we need accounts which did not close within the 01/22-06/22 time period. However, they also must not have closed within the 30-days following our start/end dates, since this propensity model measures propensity to close within 30 days. As such, the filtering date is extendend to 06/30/2022 + 30 days (32 days to be safe).
* The timeframe has been chosen for completeness: aggregate window sliding by subsequent time period to avoid peripheral outlier aggregation (32-days)
* We are not interested in accounts which simply opened during this timeframe i.e. status changed from 'Approved' to 'Active'.  Hence the phrase 'not closed accounts'
* Per our formal definition active or open accounts will have had transactions in the lookback period.
* We can take our list of acounts for a given lookback period and select from these only accounts which had [member initiated] transactions for the lookback period to ensure we arrive at **active acounts, only**

# Snowpark DataFrame Implementation for Determining Open Accounts
#### Snowflake DataFrame construct
https://docs.snowflake.com/en/developer-guide/snowpark/python/working-with-dataframes#label-snowpark-python-dataframe-construct
#### Snowflake efficiency
https://docs.snowflake.com/en/user-guide/querying-persisted-results

"When a query is executed, the result is persisted (i.e. cached) for a period of time. At the end of the time period, the result is purged from the system."

### Distributed implementation via Snowflake
Here the desire is to find accounts w/ recent transactions instead of relying on the 'ACT' status code found w/in `ACCSTATCD` as we would also find accounts which had been recently opened and have no transactions for the lookback period.  

We use the list of closed accounts to filter out accounts from RTXN.

In [5]:
def get_closed_accounts_for_timeframe(session, begin, end, lim=200):
    """
    This function represents a Snowflake-Snowpark implementation to find closed accounts for a given look-back period.  
    
    Accounts w/ the status code 'CLS' (OSIBANK-DNA system code for closed) for the given timeframe are found.
    These accounts are then aggregated by account number where the most recent/highest 'TIMEUNIQUEEXTN' column value
    is determined for an account via the aggregate function 'max'. The most recent [unique] value for the period 
    is needed as it's possible to encounter accounts which were closed but subsequently re-opened in the given period.
    'TIMEUNIQUEEXTN' represents a unique system key generated for account status changes.
    After this query is carried out then a join is performed on accounts to pull full rows of data matching 'TIMEUNIQUEEXTN'.
    
    Parameters:
        session (Snowflake session object): used to establish a connection to the desired Snowflake instance.
        begin (string: MM/DD/YYYY): a date which specifies the beginning of the lookback period inclusively.
        end (string: MM/DD/YYYY): a date which specifies the end of the lookback period inclusively.
        lim (int): an integer which specifies a query return result limit, a row return max.
        
    Returns:
        Snowflake DataFrame: A DataFrame containing the closed account information.
    """    
    # session.table() creates a Snowflake DataFrame from a table, view or stream
    df_table = session.table("ACCTACCTSTATHIST")
    
    # get accounts which match ultimate/final account status 'CLS'
    latest_status = (df_table
                     .filter((col('ACCTSTATCD') == 'CLS'))
                     .group_by('ACCTNBR')
                     .agg(max_(col('TIMEUNIQUEEXTN')).alias('TIMEUNIQUEEXTN'))
                    )
    
    # merge latest_status back to ACCTACCTSTATHIST DataFrame created from VIEW
    return (latest_status
            .join(
                df_table, 
                on='TIMEUNIQUEEXTN', 
                how='inner', 
                lsuffix='_'
            )
            .select(
                col('ACCTNBR'), 
                col('ACCTSTATCD'), 
                col('TIMEUNIQUEEXTN'), 
                col('EFFDATETIME')
            )
            .filter((f.to_date('EFFDATETIME') >= begin) & (f.to_date('EFFDATETIME') <= end))
            .limit(lim)
           )


def get_completed_active_accounts_for_timeframe(session, closed_account_snowflake_df, begin, end, lim=200):
    """
    This function represents a Snowflake-Snowpark implementation for finding open or active accounts 
    for the given look-back period where active accounts are defined as having at least 30-completed transactions
    in the specified period, and do not have a final closed disposition.
    
    Accounts with transaction posting dates for the given range are filtered from the transactions "RTXN" VIEW 
    followed by filtering for completed transactions only as determined by column "CURRRTXNSTATCD" value "C". Next
    filtering for accounts w/ greater than or equal to 30 transactions occurs to ensure enough transactions 
    for our sampling.
    This results in all account numbers with transactions, meeting our conditions, for the lookback period.
    
    After this query is carried out then a set subtraction is performed w.r.t. closed accounts using the column 'ACCTNBR'
    leaving accounts w/ >= 30 completed transactions which were also not ultimately closed w/in the lookback period.
    
    Parameters:
        session (Snowflake session object): used to establish a connection to the desired Snowflake instance.
        closed_account_snowflake_df (Snowflake DataFrame): This is a Snowflake DataFrame representing closed accounts.
        begin (string: MM/DD/YYYY): a date which specifies the beginning of the lookback period, inclusively.
        end (string: MM/DD/YYYY): a date which specifies the end of the lookback period, inclusively.
        lim (int): an integer which specifies a row return max.
        
    Returns:
        Snowflake DataFrame: A DataFrame containing accounts defined by more than 30 transactions for the lookback period
        which were also not ultimately closed ("CLS" status code) during said period.
    """      
    # we may have reversed transactions, however, we're filtering/limiting rows using 'ORIGPOSTDATE' & then 'C' for completed
    # helpful related SQL query: # SELECT ACCTNBR, COUNT(*) FROM RTXN GROUP BY ACCTNBR HAVING COUNT(*) > 30
    open_rtxn = (session
                 .table("RTXN")
                 .filter(
                     (f.to_date('ORIGPOSTDATE') >= begin) & (f.to_date('ORIGPOSTDATE') <= end)
                 )
                 .filter(col("CURRRTXNSTATCD") == 'C')  # get completed transactions, only. [SEE VIEW RTXNSTAT, col RTXNSTATCD]
                 .group_by(col("ACCTNBR")).agg(count_('ACCTNBR').alias('ACCTNBR_COUNT'))
                 .filter(col('ACCTNBR_COUNT') >= 30)    # get only accounts w/ more than 30 transactions 
                 .select('ACCTNBR')
                )
    
    # subtract out closed accounts from all open accounts --> gives same answer/outcome as a leftanti join
    # https://docs.snowflake.com/developer-guide/snowpark/reference/python/api/
    #     snowflake.snowpark.DataFrame.subtract.html#snowflake.snowpark.DataFrame.subtract
    
    return open_rtxn.subtract(closed_account_snowflake_df.select(col("ACCTNBR")))

In [6]:
%%capture
# THE FOLLOWING OUTPUT IS WITHELD DUE TO SENSITIVE Personally identifiable information (PII)

clsd_acct_sdf = get_closed_accounts_for_timeframe(snow_session, begin='1/1/2022', end='8/1/2022', lim=None)
# print(f"total number of closed accounts: {clsd_acct_sdf.count():,}")
pd.DataFrame(clsd_acct_sdf.collect())

In [7]:
%%capture
# THE FOLLOWING OUTPUT IS WITHELD DUE TO SENSITIVE Personally identifiable information (PII)

act_accts = get_completed_active_accounts_for_timeframe(
    snow_session, 
    clsd_acct_sdf, 
    begin='1/1/2022', 
    end='8/1/2022', 
    lim=None)

# print(f"total number of active acounts 1/1/2022 - 8/1/2022: {act_accts.count():,} accounts")
print("\nactive accounts w/ >= 30  completed ['C'] transactions 1/1/2022 - 8/1/2022:")
# pd.DataFrame(act_accts.collect())
print("act_accts.count():", act_accts.count())
print("act_accts.distinct().count():", act_accts.distinct().count())

act_accts.show()

## Join active accounts for window w/ ACCT table
We previously found just the account number, however, we would like all related account info for these specific accounts

In [8]:
%%capture
# THE FOLLOWING OUTPUT IS WITHELD DUE TO SENSITIVE Personally identifiable information (PII)

df_ACCTACCTSTATHIST = snow_session.table("ACCTACCTSTATHIST")
    
# get earliest 'TIMEUNIQUEEXTN' [system assigned primary key] per account
earliest_status = (
    df_ACCTACCTSTATHIST
    .group_by('ACCTNBR')
    .agg(min_(col('TIMEUNIQUEEXTN')).alias('TIMEUNIQUEEXTN'))
)

print('earliest_status:')
earliest_status.show()

act_accts_addtl = (
    earliest_status
    .join(    # get min TIMEUNIQUEEXTN for all active accounts from lookback window
        act_accts, 
        on='ACCTNBR', 
        how='inner', 
        lsuffix='_'
    )
    .join(    # get EFFDATETIME for min TIMEUNIQUEEXTN
        df_ACCTACCTSTATHIST,
        on='TIMEUNIQUEEXTN',
        how='inner',
        lsuffix='_'
    )
    .select(    # comment this out to check raw table return results
        col('ACCTNBR'), 
        col('EFFDATETIME')
    )
)

print('earliest_status after joining w/ active accounts:')
print(act_accts_addtl.count())
act_accts_addtl.show()

### Instantiate/establish new Snowflake session for writing open accounts to Snowflake table
This keeps the transactional data separate.  

In [9]:
# instantiate/establish snowflake session
snow_session_ots_innov = create_snowpark_session()

# set database & schema to be used for session
snow_session_ots_innov.use_database("OTS_INNOVATION_DB")
snow_session_ots_innov.use_schema("JOEL_C")

get_session_info(snow_session_ots_innov)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...

account        "otscuso"
role           "OTS_INNOVATION"
warehouse      "OTS_INNOVATION_WH"
database       "OTS_INNOVATION_DB"
schema         "JOEL_C"



### Write open accounts to Snowflake table under OTS_INNOVATION_DB.JOEL_C.SAMPLE_ACCOUNTS

In [10]:
%%capture
# THE FOLLOWING OUTPUT IS WITHELD DUE TO SENSITIVE Personally identifiable information (PII)

# we need to use a different snowflake session which references the new DB

pd_df = act_accts_addtl.to_pandas()
pd_df['EFFDATETIME'] = pd_df['EFFDATETIME'].dt.tz_localize('UTC')

# pd_df = pd_df['ACCTNBR'].to_frame()
pd_df['acct_open'] = 1
pd_df['effective_date'] = pd_df['EFFDATETIME'].dt.date
pd_df['window_start'] = pd.to_datetime('1/1/2022')
pd_df['window_end'] = pd.to_datetime('8/1/2022')

# code below overcomes known datetime issue when writing a Pandas DataFrame to a Snowpark DataFrame
pd_df['window_start'] = pd_df['window_start'].dt.tz_localize('UTC').dt.date
pd_df['window_end'] = pd_df['window_end'].dt.tz_localize('UTC').dt.date

# pd_df.drop(['EFFDATETIME'], axis=1, inplace=True)
pd_df = pd_df[['ACCTNBR', 'acct_open', 'effective_date', 'window_start', 'window_end']]

display(pd_df)

(
    snow_session_ots_innov    
    .create_dataframe(pd_df)
    .write.mode("append")
    .save_as_table("SAMPLE_ACCOUNTS")
)

### Check write results
Snowflake SQL is used
* IMPORTANT NOTE: The output below is the original output prior to subsampling due to a roughly 3:1 ratio of open vs closed accounts

In [26]:
output_df = pd.DataFrame(snow_session_ots_innov.sql("""
SELECT 
    ACCT_OPEN, 
    COUNT(ACCT_OPEN),
    (COUNT(ACCT_OPEN)* 100 / (Select COUNT(*) FROM OTS_INNOVATION_DB.JOEL_C.SAMPLE_ACCOUNTS)) AS PERCENT
FROM OTS_INNOVATION_DB.JOEL_C.SAMPLE_ACCOUNTS 
GROUP BY ACCT_OPEN;
""").collect())

output_df.style.hide_index()

C:\Users\jcorley\AppData\Local\Temp\ipykernel_13520\2490020236.py:10: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  output_df.style.hide_index()


ACCT_OPEN,COUNT(ACCT_OPEN),PERCENT
0,59569,24.311397
1,185456,75.688603


# Close Session

In [11]:
snow_session.close()

# LESSONS LEARNED

* A single Snowflake session object can be used as there is no need for multiple sessions.  In this case I instantiated a session to work with the datatbase tables and then created another separate session to write my open account results back to Snowflake.  
    - This would however require changing the table, schema, etc. for the existing session object
* While not alien to me the functional programming paradigm was not my preference at the onset of this internship, however, I would quickly develope an aptitude for this paradigm and in time enjoy using it.
    - functional programming in this case would follow SQL statements and so I would use SQL as a stepping stone for structuring my functional programming code

# Older But Related Analysis/Approach

## Dormant & Inactive Accounts Analysis
This approach used activity codes in an attempt to determined open accounts. However, this is a naive approach as an account may have closed, but then subsequently reopened during the lookback period. 

In [ ]:
df_table = snow_session.table("ACCTACCTSTATHIST")

# NOTE: CWB comprises a significantly smaller number of of overall accounts vs CLS or Chargeoff
total_opn = df_table.where(
    (col('ACCTSTATCD') == 'ACT') | (col('ACCTSTATCD') == 'DORM') | (col('ACCTSTATCD') == 'IACT')
).count()
total_act = df_table.where(col('ACCTSTATCD') == 'ACT').count()

print(f"Total number of rows returned: {total_opn:,}")
print(f"% ACT = {total_act / total_opn:.4%}\n")

# .show() returns the 1st 10 rows vs .collect() would return all rows
df_table.filter((col('ACCTSTATCD') == 'ACT') | (col('ACCTSTATCD') == 'DORM') | (col('ACCTSTATCD') == 'IACT')).show()

A high number of accounts, while open, are either inactive (no activity within 1-year) or dormant (no activity within 2-years); (combined: 18.3735%).  

Since inactive or dormant accounts will not have activity within the last 15-30-days by definition these accounts can be ommitted.

In [ ]:
# limit our returned number of rows to 10,000 vs 12.6+ million
view_obj = get_view(snow_session, view="ACCTACCTSTATHIST", row_lim=1000, printSession=True)
view_df = pd.DataFrame(view_obj)
# print(view_df.columns.to_list())    # print column headers if needed for larger VIEWs

# does capture pattern of ORIG --> APPR --> ACT
# view_df.sort_values(['ACCTNBR', 'EFFDATETIME']).head(20)

# capture more recent accounts from 2023 where more than 1 member transaction occurred
view_df_filter = view_df[view_df.duplicated('ACCTNBR', False)].sort_values(['ACCTNBR', 'EFFDATETIME'])
# view_df_filter[view_df_filter['EFFDATETIME'].dt.year == 2023].tail(20)
view_df_filter.tail(20)

Account approval `APPR` appears to precede an account becoming active within the system `ACT`.

# Views

## `ACCTSTAT`

This VIEW effectively **provides a key for account status types** via columns `ACCTSTATCD` & `ACCTSTATDESC`

Our formal definition of open accounts includes accounts which have had recent transactions w/in the lookback period.
*For this VIEW exploration, only, open accounts are interpreted to mean accounts with the following status:*
- **Active** - recent member activity has occurred under the account not meeting the above thresholds for inactivity nor dormancy.

*Other important observations*
- **Approved** - accounts include loans (per this system--see VIEW ) and this may indicate when loan accounts or transactional (checking, savings, etc.) accounts were approved; but not necessarily opened.  
- **Originating** - this is likely *when* the transactional account or loan paperwork was signed by the member and is found for all open as well as closed accounts (originating as in orgination).
- **Inactive** - likely indicates an account was opened but has not had recent member activity, as defined by no member activity within a 365-day threshold (vs Closed) [SEE `MJMIACCTTYP` below].  
    * While the account is open, it will not have had activity occurring with 1-year and so by definition we can exclude inactive accounts for a 15-30-day look back.
- **Dormant** - likely indicates an account was opened but has not had any member account activity within a 730-day (2-years) threshold [SEE `MJMIACCTTYP` below].  
    * While the account is open, it will not have had activity occurring with 2-years and so by definition we can exclude dormant accounts for a 15-30-day look back.


In [5]:
pd.DataFrame(
    get_view(snow_session, view="ACCTSTAT", row_lim=None, printSession=False)
)[['ACCTSTATCD', 'ACCTSTATDESC', 'DATELASTMAINT']]    # grab important columns, only, from underlying VIEW


records returned from VIEW=ACCTSTAT: 11


,ACCTSTATCD,ACCTSTATDESC,DATELASTMAINT
0,APPR,Approved,1994-01-01 00:00:00
1,CLS,Closed,1994-01-01 00:00:00
2,CO,Chargeoff,1994-01-01 00:00:00
3,CWB,Closed with Balances Remaining,1994-01-01 00:00:00
4,DORM,Dormant,1994-01-01 00:00:00
5,IACT,Inactive,1994-01-01 00:00:00
6,ORIG,Originating,1994-01-01 00:00:00
7,ACT,Active,1994-01-01 00:00:00
8,NPFM,Non-Accrual,1996-04-22 11:37:41
9,DENI,Loan Denied,1995-12-28 12:45:40


## `ACCTTYP`
Provides a key for determining the type of account

In [6]:
pd.DataFrame(get_view(snow_session, view="ACCTTYP", row_lim=None, printSession=False))


records returned from VIEW=ACCTTYP: 11


,ACCTTYPCD,ACCTTYPDESC,DATELASTMAINT
0,0,Funding/None,1997-11-19 17:30:59
1,8,Utility,1997-11-19 17:30:59
2,9,Other,1997-11-19 17:30:59
3,10,Savings Account,1997-11-19 17:30:59
4,20,Checking Account,1997-11-19 17:30:59
5,30,Credit Account,1997-11-19 17:30:59
6,50,Other (50),1997-11-19 17:34:59
7,90,Loan,2006-07-16 09:15:55
8,91,Mortgage,2006-07-16 09:15:55
9,92,Consumer/Installment Loan,2006-07-16 09:15:55


## `MJMIACCTTYP`

### Determine Threshold Time Limit Required for Inactive Account
Inactive accounts appear to have a threshold of 1-year from last member activity per the given VIEW

In [7]:
activity = pd.DataFrame(get_view(snow_session, view="MJMIACCTTYP", row_lim=None, printSession=False))

activity[activity['INACTIVEDAYS'].notnull()][['MJACCTTYPCD', 'MIACCTTYPCD', 'MIACCTTYPDESC', 'INACTIVEDAYS']]


records returned from VIEW=MJMIACCTTYP: 515


,MJACCTTYPCD,MIACCTTYPCD,MIACCTTYPDESC,INACTIVEDAYS
151,CK,BFBC,Free Business Checking,365.0
158,CK,BZBA,Business Zero Balance Account,365.0
215,CK,BBCP,Business Checking Plus,365.0
319,SAV,PART,PCA - INTERNAL USE ONLY,365.0


In [8]:
print(len(activity.columns.values), activity.columns.values)
activity

139 ['MJACCTTYPCD' 'MIACCTTYPCD' 'MIACCTTYPDESC' 'PMTSPDCD' 'SLPCD'
 'PMTMETHCD' 'PMTCALPERIODCD' 'BILLINGLEADDAYS' 'RPTDEFERINTYN'
 'EXCESSTXNYN' 'PASSBOOKYN' 'CHECKSYN' 'TXNACCTYN' 'CANDRAWFROMYN'
 'STMTYN' 'COLLINTYN' 'MINLOANAMT' 'MAXLOANAMT' 'MAXOVERDRAFTAMT'
 'MINDRAWAMT' 'MINSWEEPAMT' 'RNEWGRACEDAYS' 'ADDTDEPYN' 'ADDTDEPGRACEDAYS'
 'ADDTDEPMATEXTENYN' 'RNEWMIACCTTYPCD' 'MINTERMDAYS' 'MAXTERMDAYS'
 'DEFAULTTERMDAYS' 'GRACEDAYS' 'DEFAULTDUEDAY' 'DATELASTMAINT'
 'REVOLVELOANYN' 'DISPLAYSEQNBR' 'MAXCOUPONCOUNT' 'INACTIVEDATE' 'VALIDYN'
 'EFFDATE' 'RCVBGENMETHCD' 'PMTOVRYN' 'BILLADVANCEYN'
 'PREPAYMENTALLOWEDYN' 'DEFAULTCREDITLIMIT' 'DRAWINCRAMT' 'PRIMARYSTMTYN'
 'SECONDARYSTMTYN' 'SWEEPFROMYN' 'SWEEPTOYN' 'SWEEPMGMTYN'
 'DRAWBALGOALAMT' 'ACCTOVRPSBKYN' 'RATECHANGELEADDAYS' 'PMTCHANGELEADDAYS'
 'MINBAL' 'MINDEPAMT' 'MINWTHAMT' 'RPT1098YN' 'RPTPTSYN'
 'ACCRTHRUDUEDATEYN' 'DEMANDYN' 'LETTEROFCREDITTYPCD' 'LOANLIMITYN'
 'ADDTDISBYN' 'CREDITREPORTTYPCD' 'CUSHIONMONTHS' 'PARTIALPMTALLOWED

,MJACCTTYPCD,MIACCTTYPCD,MIACCTTYPDESC,PMTSPDCD,SLPCD,PMTMETHCD,PMTCALPERIODCD,BILLINGLEADDAYS,RPTDEFERINTYN,EXCESSTXNYN,...,ALLOWSUBMASTERLINESYN,REPRICPLANALLOWEDYN,DELIVERYMETHCD,ALLOWACCTPPAYVALPAYEENAMEYN,ESCHEATMENTPROPTYPCD,MULTIADVOPTLINEYN,RPTMJACCTTYPCD,NONACCRUALTOACTDAYS,RQDNBRCONSECONTIMEPMTSTOACT,PK_KEY
0,GL,GL,General Ledger,None,None,None,None,NaN,N,N,...,N,N,None,N,None,N,None,None,None,bae466cd591932697b8d69b012de5cbd
1,RR,RR,Regulatory Reporting,None,None,None,None,NaN,N,N,...,N,N,None,N,None,N,None,None,None,a8b9a855c2bb08d09b7b2463db70f820
2,RTMT,IRA,IRA Plan,None,None,None,None,NaN,N,N,...,N,N,None,N,None,N,None,None,None,964f970e30db4fc2449dbde67c4eedf8
3,BKCK,MO,Money Order Arvada Ridge,None,None,None,None,NaN,N,N,...,N,N,None,N,None,N,None,None,None,2c6f636f0507acb739d30a5f19660252
4,RR,BOND,U S Bonds Redeemed,None,None,None,None,NaN,N,N,...,N,N,None,N,None,N,None,None,None,7253fe9ef8b46c96d724b263bd618571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,CNS,FLNL,CUDC FL New Lease,HORZ,None,NONE,MNTH,31.0,N,N,...,N,N,PRNT,N,None,N,None,None,None,2d66ccc2e2580b0c08655317a3290475
511,CNS,ZAZL,Z C/O - AZNL - AZ New Lease,HORZ,None,NONE,MNTH,31.0,N,N,...,N,N,PRNT,N,None,N,None,None,None,6190552e39d8ed45867203765058d304
512,CML,CT20,CML RE 1st Sec Term Loan 20yr,HORZ,None,NONE,MNTH,31.0,N,N,...,N,N,PRNT,N,None,N,None,None,None,aacaae8efb5229cdc10ce7ce85a4e2dd
513,SAV,UPSV,Upgrade Deposits,None,None,None,None,NaN,N,N,...,N,N,PRNT,N,None,N,None,None,None,82ae8b21354114ee3f2ca04156689459


### Determine Threshold Time Limit Required for Dormant Account
Apart from several unique cases the dormant account threshold time limit is 730 days (2-years) since last member activity per the given view for accounts w/ a threshold

### `AACTAACTSTATHIST`

The specific VIEW `AACTAACTSTATHIST` captures salient/important account status information: 
- `ACCSTATCD` - account status (Active, Closed, etc.) 
- `EFFDATETIME` - effective datetime of account status change. **i.e. the provided datetime when an account was closed**
- `ACCTNBR` - account number

BELLCO's Snowflake database has been chosen here for the sake of simplicity, however, so long as the other CU's databases follow the same naming convention and structure they may be accessed similarly.